<a href="https://colab.research.google.com/github/Rahulwakalkar/Text_extraction_from_Image/blob/master/Invoice_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [9]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [14]:
import re
import json

# Open and read the text file
with open('/content/drive/My Drive/PML/extracted_data.json', 'r', encoding='utf-8') as file:
    text = file.read()

# Define regular expressions for extraction
invoice_number_pattern = r"Serial No\. of Invoice:\s*(\S+)"
invoice_date_pattern = r"Date of Invoice :\s*(\d{2}-\d{2}-\d{4})"
gst_number_pattern = r"GSTIN:\s*(\S+)"
vendor_address_pattern = r"URJAYANT ENGINEERING PVT LTD\n(.+?)\n"
buyer_name_pattern = r"Name:\s*(.+)"
buyer_address_pattern = r"Address : (.+?)\nContact No:"

item_details_pattern = r"(\d+)\s+\|(.+?)\s+(\d+)\s+(.+)\s+(\d+\.\d+)\s+(\d+\.\d+)"
total_invoice_pattern = r"Invoice Total\s*\n([\d,]+)\s*[\w\s%]+\s*([\d,]+)\s*[\w\s%]+\s*([\d,]+)"

# Extract information using regular expressions
invoice_number_match = re.search(invoice_number_pattern, text)
invoice_number = invoice_number_match.group(1) if invoice_number_match else None

invoice_date_match = re.search(invoice_date_pattern, text)
invoice_date = invoice_date_match.group(1) if invoice_date_match else None

gst_number_match = re.search(gst_number_pattern, text)
gst_number = gst_number_match.group(1) if gst_number_match else None

vendor_address_match = re.search(vendor_address_pattern, text)
vendor_address = vendor_address_match.group(1) if vendor_address_match else None

buyer_name_match = re.search(buyer_name_pattern, text)
buyer_name = buyer_name_match.group(1) if buyer_name_match else None

buyer_address_match = re.search(buyer_address_pattern, text)
buyer_address = buyer_address_match.group(1) if buyer_address_match else None

# Extract item details using regular expressions
item_details = re.findall(item_details_pattern, text)
item_details = [
    {"Code": code.strip(), "Description": desc.strip(), "Quantity": qty, "Unit": unit, "Rate": rate, "Total": total}
    for code, desc, qty, unit, rate, total in item_details
]

# Extract total invoice amount and tax amounts
total_invoice_match = re.search(total_invoice_pattern, text)
total_invoice_amount = total_invoice_match.group(1).replace(',', '') if total_invoice_match else None
total_tax_amount = total_invoice_match.group(3).replace(',', '') if total_invoice_match else None

# Create a dictionary with the extracted information
extracted_info = {
    "Invoice Number": invoice_number,
    "Invoice Date": invoice_date,
    "GST Number": gst_number,
    "Vendor Address": vendor_address,
    "Buyer Name": buyer_name,
    "Buyer Address": buyer_address,
    "Item Details": item_details,
    "Total Invoice Amount": total_invoice_amount,
    "Total Tax Amount": total_tax_amount
}

# Save the extracted information as a JSON file
output_filename = "structured_data1.json"
with open(output_filename, "w", encoding="utf-8") as json_file:
    json.dump(extracted_info, json_file, ensure_ascii=False, indent=4)

print(f"Extracted data saved to {output_filename}")


Extracted data saved to structured_data1.json


In [13]:
# Create a DataFrame
import pandas as pd

data = {
    "Invoice Number": [invoice_number],
    "Invoice Date": [invoice_date],
    "GST Number": [gst_number],
    "Vendor Address": [vendor_address],
    "Buyer Name": [buyer_name],
    "Buyer Address": [buyer_address]
}
df = pd.DataFrame(data)

# Add item details to the DataFrame
item_df = pd.DataFrame(item_details, columns=["Item Code", "Item Description", "Quantity", "Unit", "Rate", "Total"])
df = pd.concat([df, item_df], axis=1)

# Add total invoice amount to the DataFrame
df["Total Invoice Amount"] = total_invoice_amount

# Print the DataFrame
df

,Invoice Number,Invoice Date,GST Number,Vendor Address,Buyer Name,Buyer Address,Item Code,Item Description,Quantity,Unit,Rate,Total,Total Invoice Amount
0,U-1/19-20/947\nDate,13-08-2019,Purchase,None,EFACEC INDIA PVT LTD Name: EFACEC INDIA PVT LT...,None,NaN,NaN,2,A\n\nCGST Total ( In Words) :\nRupees Seven Th...,6.80,340.00,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,A\n\nCGST Total ( In Words) :\nRupees Seven Th...,6.80,340.00,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8538,59.00,213.00,12567.00,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6116,"5%| 100 Prs 38.00) Prs 3,800.00\n~~ eGst 95.00...",95.00,190.00,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Kgs :\nBilled To : EFACEC INDIA P LTD | Shippe...,9000.00,810.00,None
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,32601131 CAPACITIVE INSULATOR WITH 85389000 60...,961.00,1.00,None
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121003,"elitcenr hicis 7\n|Roa8 Block 6-1, Moan Coovet...",432.00,864.00,None
